# 2.4 Textlicher Teil Bebauungsplan – Embeddings + Vector Store (GPT-4o)

Das Context-Window (128.000 Tokens) in GPT-4o ist begrenzt. Entsprechend muss ab einer gewissen Seitenanzahl des schriftlichen Dokuments eines Bebauungsplans auf Emebddings zurückgegriffen werden. (~765 Tokens pro Bild mit hoher Auflösung / 85 Tokens pro Bild mit niedriger Auflösung)

IDEE: Embeddings dienen ausschließlich dazu die Seitenanzahl des schriftlichen Dokuments zu reduzieren. GPT-4o bekommt anschließend wieder die Bilder als Kontext zu sehen. Dadurch gehen möglichst wenig Informationen verloren.I

* Zero-Shot
* Chain-Of-Thought

In [7]:
from utils.pdf2prompts import pdf2prompts
import asyncio
import os
from dotenv import load_dotenv, find_dotenv
from langchain_openai import ChatOpenAI
from langchain.schema.messages import AIMessage, HumanMessage
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

load_dotenv(find_dotenv())

model = ChatOpenAI(
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    model_name="gpt-4o",
    model_kwargs={"top_p": 0, "seed": 42},
    temperature=0,
)

In [ ]:
# A) OCR via RapidOCR --> CNN/RNN != Transformer (LLM)
# from langchain_community.document_loaders import PyMuPDFLoader
# pdf_path = "../data/raw/bpläne/2_zeichnung_textteil_getrennt/F 11- 02 Gewerbegebiet Himmelreich - schriftlicher Teil.pdf"
# loader = PyMuPDFLoader(pdf_path, extract_images=True)
# data = loader.load()
# print(data[0])

In [8]:
# OCR via GPT-4o / Kompletter schriftlicher Teil
# Idee: Jede Seite einzeln verarbeiten, um den Fokus zu maximieren und Context-Window klein zu halten.
async def page2text(page_prompt):
    msg = model.invoke([
        AIMessage(
            content=[
                {
                    "type": "text",
                    "text": "Du bist ein Assistent um Textinformationen aus dem schriftlichen Teil eines Bebauungsplans zu extrahieren."
                },
            ]
        ),
        HumanMessage(
            content=[
            {
                "type": "text",
                "text": 'Extrahiere den kompletten Textinhalt.'
            },
            page_prompt
            ]
        ),
    ])
    return msg.content

pdf_path = "../data/raw/bpläne/2_zeichnung_textteil_getrennt/F 11- 02 Gewerbegebiet Himmelreich - schriftlicher Teil.pdf"
prompts = pdf2prompts(pdf_path)
data_loaders = list(map(lambda prompt: page2text(prompt), prompts))
data = await asyncio.gather(*data_loaders)
print(data[0])

SCHRIFTLICHER TEIL (Teil B)

BEBAUUNGSPLAN "GEWERBEGEBIET HIMMELREICH"

STADT LAICHINGEN, GEMARKUNG FELDSTETTEN, ALB-DONAU-KREIS

Der Geltungsbereich wird durch das Planzeichen im Lageplan begrenzt.

Lageplan M 1: 500

Für die planungsrechtlichen bzw. bauordnungsrechtlichen Festsetzungen gelten:

- Baugesetzbuch (BauGB)
  in der Fassung der Bekanntmachung vom 27.08.1997 (BGBl. I. S. 2141).

- Baunutzungsverordnung (BauNVO)
  in der Fassung der Bekanntmachung vom 23.01.1990 (BGBl. S. 132), zuletzt geändert am 22.04.1993 (BGBl. I. S. 466).

- Planzeichenverordnung 1990 (PlanzV 90)
  in der Fassung der Bekanntmachung vom 18.12.1990 (BGBl. I. S. 58).

- Landesbauordnung (LBO)
  in der Fassung der Bekanntmachung vom 08.08.1995 (GBl. S. 617).

Bisherige Festsetzungen:
Mit in Kraft treten dieses Bebauungsplanes treten im Geltungsbereich alle bisherigen gemeindlichen bauplanungsrechtlichen und bauordnungsrechtlichen Festsetzungen außer Kraft.


In [ ]:
# A) Create Embeddings (B-Plan) --> Chunk-Size = Page


In [ ]:
# B) Create Vector Store

In [6]:
# C) Query Vector Store ==> Relevant Pages

In [ ]:
# D) Connect Informations via Chat
# * Add relevant pages to context-window